In [2]:
#@title Download Language Model
!python -m spacy download en_core_web_lg
!python -m spacy download fr_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 960.6 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.8/571.8 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
#@title Load CSV
import pandas as pd

df = pd.read_csv("../../data/ri_annotated_texts_filtered.csv")
df

,Version initiale,Version retraitée,Stratégie
0,Dispositif d'apprentissage du français : perme...,Des ateliers 2 fois par semaine pour progresse...,Complement
1,Dispositif d'apprentissage du français : perme...,Des ateliers 2 fois par semaine pour progresse...,Complement
2,Dispositif d'apprentissage du français : perme...,"Des ateliers pour progresser en français, mieu...",SynSemantic
3,Dispositif d'apprentissage du français : perme...,"Des ateliers pour progresser en français, mieu...",Compression
4,Dispositif d'apprentissage du français : perme...,"Des ateliers pour progresser en français, mieu...",Syntactic
...,...,...,...
379,Suivi socio-administratif : individualisé visa...,"Conseils pour trouver une garde d'enfant, un m...",Modulation
380,Suivi socio-administratif : individualisé visa...,"Conseils pour trouver une garde d'enfant, un m...",SynSemantic
381,Suivi socio-administratif : individualisé visa...,"Conseils pour trouver une garde d'enfant, un m...",Explanation
382,Suivi socio-administratif : individualisé visa...,"Conseils pour trouver une garde d'enfant, un m...",Examples


In [4]:
#@title Setup Presidio
from presidio_analyzer import AnalyzerEngine
from presidio_analyzer.nlp_engine import NerModelConfiguration, SpacyNlpEngine
from presidio_anonymizer import AnonymizerEngine

model_config = [
    {"lang_code": "en", "model_name": "en_core_web_lg"},
    {"lang_code": "fr", "model_name": "fr_core_news_lg"},
]

nlp_engine = SpacyNlpEngine(models=model_config, ner_model_configuration=NerModelConfiguration(labels_to_ignore=["MISC"]))

analyzer_engine = AnalyzerEngine(
    nlp_engine=nlp_engine,
    supported_languages=["en", "fr"]
)
anonymizer_engine = AnonymizerEngine()

In [5]:
#@title Establish Presidio Anonymization Config
from faker import Faker
from presidio_anonymizer.entities import OperatorConfig

fake = Faker("fr_FR")

# Cache structure to store mappings for each entity
cache = {
    "PERSON": {},
    "PHONE_NUMBER": {},
    "EMAIL_ADDRESS": {}
}

def reset_cache():
    global cache
    cache = {
        "PERSON": {},
        "PHONE_NUMBER": {},
        "EMAIL_ADDRESS": {}
    }

def get_cached_value(entity_type, x):
    if x in cache[entity_type]:
        return cache[entity_type][x]
    else:
        # Generate appropriate fake value based on entity type
        if entity_type == "PERSON":
            fake_value = fake.name()
        elif entity_type == "PHONE_NUMBER":
            fake_value = fake.phone_number()
        elif entity_type == "EMAIL_ADDRESS":
            fake_value = fake.safe_email()
        else:
            return x

        # Store the generated value in the cache
        cache[entity_type][x] = fake_value
        return fake_value

# Configure the anonymization operators using the cached values
anonymization_config = {
    "PERSON": OperatorConfig(
        "custom",
        {"lambda": lambda x: get_cached_value("PERSON", x)}
    ),
    "PHONE_NUMBER": OperatorConfig(
        "custom",
        {"lambda": lambda x: get_cached_value("PHONE_NUMBER", x)}
    ),
    "EMAIL_ADDRESS": OperatorConfig(
        "custom",
        {"lambda": lambda x: get_cached_value("EMAIL_ADDRESS", x)}
    ),
}

In [6]:
cache["EMAIL_ADDRESS"].values()

dict_values([])

In [7]:
#@title Utility Functions
def anonymize_text(text):
    if not isinstance(text, str):  # Handle non-string values gracefully
        return text
    analyzer_results = analyzer_engine.analyze(text, entities=["PERSON", "PHONE_NUMBER", "EMAIL_ADDRESS"], language="fr")
    anonymized_text = anonymizer_engine.anonymize(text, analyzer_results, anonymization_config)
    return anonymized_text.text

In [8]:
#@title Test Anonymization
text_with_pii = """
Permanence d'inscription tous les mardis de 16h à 17h15 avec Marine Perrau au 17 avenue Charles de Gaulle à Paris. Sans rendez vous, renseignements par mail : toto@gaaa.fr , par téléphone ou SMS : 01.02.03.04.05
"""
anonymize_text(text_with_pii)

"\nPermanence d'inscription tous les mardis de 16h à 17h15 avec Thibaut Vaillant au 17 avenue Aimée Voisin-Launay à Paris. Sans rendez vous, renseignements par mail : bernardbazin@example.net , par téléphone ou SMS : 07.89.34.01.97\n"

In [9]:
#@title Apply Anonymization
df["Version initiale"] = df["Version initiale"].apply(anonymize_text)
df["Version retraitée"] = df["Version retraitée"].apply(anonymize_text)
df

,Version initiale,Version retraitée,Stratégie
0,Dispositif d'apprentissage du français : perme...,Des ateliers 2 fois par semaine pour progresse...,Complement
1,Dispositif d'apprentissage du français : perme...,Des ateliers 2 fois par semaine pour progresse...,Complement
2,Dispositif d'apprentissage du français : perme...,"Des ateliers pour progresser en français, mieu...",SynSemantic
3,Dispositif d'apprentissage du français : perme...,"Des ateliers pour progresser en français, mieu...",Compression
4,Dispositif d'apprentissage du français : perme...,"Des ateliers pour progresser en français, mieu...",Syntactic
...,...,...,...
379,Suivi socio-administratif : individualisé visa...,"Conseils pour trouver une garde d'enfant, un m...",Modulation
380,Suivi socio-administratif : individualisé visa...,"Conseils pour trouver une garde d'enfant, un m...",SynSemantic
381,Suivi socio-administratif : individualisé visa...,"Conseils pour trouver une garde d'enfant, un m...",Explanation
382,Suivi socio-administratif : individualisé visa...,"Conseils pour trouver une garde d'enfant, un m...",Examples


In [10]:
#@title Save CSV
df.to_csv("../../data/ri_annotated_texts_filtered_anonymized.csv", index=False)